# Spam Email Detection for Indian Context
### With Interactive UI Interface

## Step 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# For UI - uncomment the option you want to use
# Option 1: ipywidgets (built-in to Jupyter)
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Option 2: Gradio (install with: pip install gradio)
# import gradio as gr

## Step 2: Load and Explore Data

In [ ]:
spam_df = pd.read_csv("indian_email_dataset.csv")

In [ ]:
spam_df

In [ ]:
# inspect data
spam_df.groupby('label').describe()

## Step 3: Prepare Data for Training

In [ ]:
spam_df['spam']=spam_df['label'].apply(lambda x: 1 if x == 'spam' else 0)

In [ ]:
spam_df.head(10)

## Step 4: Split Data into Training and Testing Sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(spam_df['msg'],spam_df['spam'])

## Step 5: Vectorize Text Data

In [ ]:
# Creates a count of unique word
cv = CountVectorizer()
x_train_count = cv.fit_transform(x_train.values)

In [ ]:
x_train_count.toarray()

## Step 6: Train the Model

In [ ]:
# train model
model = MultinomialNB()
model.fit(x_train_count,y_train)

## Step 7: Test Model Accuracy

In [ ]:
# test model
x_test_count = cv.transform(x_test)
accuracy = model.score(x_test_count,y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

## Step 8: Manual Testing Examples
* if prediction is **1** that means **Spam** ⚠️
* if prediction is **0** that means **Ham** (Not Spam) ✅

In [ ]:
# Test 1: Spam example
email_test = ["Your property tax Rs 2 lakhs overdue. Pay at 8765432109"]
email_test_count = cv.transform(email_test)
prediction = model.predict(email_test_count)[0]
print(f"Email: {email_test[0]}")
print(f"Prediction: {prediction} ({'SPAM' if prediction == 1 else 'HAM'})")

In [ ]:
# Test 2: Ham example
email_test = ["I HAVE A DATE ON SUNDAY WITH WILL!!"]
email_test_count = cv.transform(email_test)
prediction = model.predict(email_test_count)[0]
print(f"Email: {email_test[0]}")
print(f"Prediction: {prediction} ({'SPAM' if prediction == 1 else 'HAM'})")

---
# 🎨 INTERACTIVE UI INTERFACE
---

## UI Option 1: ipywidgets (Simple & Built-in)
**Perfect for Jupyter Notebook - No installation needed!**

In [ ]:
# Create UI components
email_input = widgets.Textarea(
    value='',
    placeholder='Enter your email text here...',
    description='Email:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px')
)

check_button = widgets.Button(
    description='Check Email',
    disabled=False,
    button_style='primary',
    tooltip='Click to check if email is spam or ham',
    icon='check',
    layout=widgets.Layout(width='200px')
)

clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='warning',
    tooltip='Clear the input',
    icon='eraser',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

# Function to check email
def check_email(b):
    with output_area:
        clear_output()
        
        email_text = email_input.value.strip()
        
        if not email_text:
            display(HTML('<div style="padding: 20px; background-color: #fff3cd; border-left: 5px solid #ffc107; border-radius: 5px;"><h3 style="color: #856404; margin: 0;">⚠️ Please enter an email to check!</h3></div>'))
            return
        
        # Transform and predict
        email_count = cv.transform([email_text])
        prediction = model.predict(email_count)[0]
        probability = model.predict_proba(email_count)[0]
        
        # Display result
        if prediction == 1:
            result_html = f'''
            <div style="padding: 20px; background-color: #f8d7da; border-left: 5px solid #dc3545; border-radius: 5px;">
                <h2 style="color: #721c24; margin-top: 0;">🚫 SPAM DETECTED!</h2>
                <p style="color: #721c24; font-size: 16px; margin: 10px 0;"><strong>Email Text:</strong> {email_text}</p>
                <p style="color: #721c24; font-size: 14px; margin: 5px 0;"><strong>Confidence:</strong> {probability[1]*100:.2f}%</p>
                <p style="color: #721c24; font-size: 14px; margin: 5px 0;">This email appears to be spam. Be cautious!</p>
            </div>
            '''
        else:
            result_html = f'''
            <div style="padding: 20px; background-color: #d4edda; border-left: 5px solid #28a745; border-radius: 5px;">
                <h2 style="color: #155724; margin-top: 0;">✅ LEGITIMATE EMAIL (HAM)</h2>
                <p style="color: #155724; font-size: 16px; margin: 10px 0;"><strong>Email Text:</strong> {email_text}</p>
                <p style="color: #155724; font-size: 14px; margin: 5px 0;"><strong>Confidence:</strong> {probability[0]*100:.2f}%</p>
                <p style="color: #155724; font-size: 14px; margin: 5px 0;">This email appears to be legitimate!</p>
            </div>
            '''
        
        display(HTML(result_html))

# Function to clear input
def clear_input(b):
    email_input.value = ''
    with output_area:
        clear_output()

# Attach functions to buttons
check_button.on_click(check_email)
clear_button.on_click(clear_input)

# Display UI
display(HTML('<h1 style="color: #2c3e50; text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; margin-bottom: 20px;">📧 Spam Email Detector</h1>'))
display(HTML('<p style="text-align: center; color: #555; font-size: 16px; margin-bottom: 20px;">Enter an email message below to check if it\'s spam or legitimate</p>'))
display(email_input)
display(widgets.HBox([check_button, clear_button], layout=widgets.Layout(justify_content='center', margin='20px 0')))
display(output_area)

### 📝 Try These Example Emails:

**SPAM Examples:**
- `FREE! Get instant loan upto Rs 1 lakh without documents. Call 8912345678`
- `Your property tax Rs 2 lakhs overdue. Pay at 8765432109`
- `Congratulations! You won iPhone 15 from Flipkart. Claim now at winner123.com`
- `BREAKING: Modi announces Rs 75000 lakh for all families. Register 8765432109`

**HAM (Legitimate) Examples:**
- `Hey, can we reschedule our meeting to 3:30 instead?`
- `Your PhonePe payment to ABC successful`
- `I HAVE A DATE ON SUNDAY WITH WILL!!`
- `Reminder: Doctor appointment tomorrow at 10:30 AM`

---
## UI Option 2: Gradio Interface (Professional & Shareable)
**Install first: `pip install gradio`**

Gradio creates a beautiful web interface that can be shared with others!

In [ ]:
# Uncomment to use Gradio (install first: pip install gradio)
"""
import gradio as gr

def predict_spam(email_text):
    if not email_text.strip():
        return "Please enter an email to check!", 0.0, ""
    
    # Transform and predict
    email_count = cv.transform([email_text])
    prediction = model.predict(email_count)[0]
    probability = model.predict_proba(email_count)[0]
    
    if prediction == 1:
        result = "🚫 SPAM DETECTED!"
        confidence = probability[1] * 100
        explanation = "This email appears to be spam. Be cautious and don't click any links or provide personal information."
    else:
        result = "✅ LEGITIMATE EMAIL (HAM)"
        confidence = probability[0] * 100
        explanation = "This email appears to be legitimate and safe."
    
    return result, confidence, explanation

# Create Gradio interface
demo = gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(
        lines=5,
        placeholder="Enter your email text here...",
        label="Email Message"
    ),
    outputs=[
        gr.Textbox(label="Prediction Result"),
        gr.Number(label="Confidence (%)"),
        gr.Textbox(label="Explanation")
    ],
    title="📧 Spam Email Detector for Indian Context",
    description="Enter an email message to check if it's spam or legitimate (ham).",
    examples=[
        ["FREE! Get instant loan upto Rs 1 lakh without documents. Call 8912345678"],
        ["Hey, can we reschedule our meeting to 3:30 instead?"],
        ["Your property tax Rs 2 lakhs overdue. Pay at 8765432109"],
        ["Reminder: Doctor appointment tomorrow at 10:30 AM"]
    ],
    theme=gr.themes.Soft()
)

# Launch the interface
demo.launch(share=False, inline=True)
"""

---
## Additional Manual Testing (Original Code)
You can still run individual tests below if needed:

In [ ]:
# Test: Spam
email_ham = ["BREAKING: Modi announces Rs 75000 lakh for all families. Register 8765432109"]
email_ham_count = cv.transform(email_ham)
model.predict(email_ham_count)

In [ ]:
# Test: Spam
email_ham = ["FREE! Get instant loan upto Rs 1 lakh without documents. Call 8912345678"]
email_ham_count = cv.transform(email_ham)
model.predict(email_ham_count)

In [ ]:
# Test: Ham
email_ham = ["Your PhonePe payment to 10:30 successful"]
email_ham_count = cv.transform(email_ham)
model.predict(email_ham_count)

In [ ]:
# Test: Ham
email_ham = ["I HAVE A DATE ON SUNDAY WITH WILL!!"]
email_ham_count = cv.transform(email_ham)
model.predict(email_ham_count)

---
## 📚 Instructions:

### For ipywidgets UI (Recommended - Easiest):
1. Run all cells from top to bottom
2. When you reach the "ipywidgets" section, you'll see an interactive text box
3. Type or paste any email text
4. Click "Check Email" button
5. See the result displayed below!

### For Gradio UI (Advanced):
1. First install: `pip install gradio`
2. Uncomment the code in the Gradio cell (remove the triple quotes)
3. Run the cell
4. A web interface will open where you can test emails
5. You can even share it with others using the `share=True` option!

---

## 🎯 Model Performance:
- **Accuracy:** The model will show its accuracy after training
- **Prediction:** 0 = Ham (Not Spam), 1 = Spam
- **Confidence:** Shows how sure the model is about its prediction

---